# Micro benchmarks for pycoalescence

Runs for a set of values of sigma, speciation rates and spatial scales.

In [13]:
import pathlib
import numpy as np
import shutil
import pandas as pd
import time

from pycoalescence import Map, Simulation

In [14]:
def create_null_tif(file: pathlib.Path, size: int):
    if file.exists():
        file.unlink()
    m = Map()
    m.data = np.ones([size, size])
    m.create(str(file))

In [15]:
temp_dir = pathlib.Path("temp")
output_dir = pathlib.Path("outputs")

In [16]:
if output_dir.exists():
    shutil.rmtree(output_dir)

### First time simulations on closed landscapes, with a fine and a coarse map

In [13]:
all_times = []
sigma = 2
task = 1
for spec_rate in [0.0000001, 0.00001, 0.001, 0.1]:
    for size in [2**x for x in range(3, 9)]:
        for scale in [3, 9, 25]:
            task += 1
            fine_map_file = pathlib.Path(temp_dir, "temp1.tif")
            coarse_map_file = pathlib.Path(temp_dir, "temp2.tif")
            create_null_tif(fine_map_file, size)
            create_null_tif(coarse_map_file, size * scale)
            offset = np.floor((scale - 1)*size/2)
            for seed in range(1, 4):
                start = time.time()
                s = Simulation(logging_level=40)
                s.set_simulation_parameters(seed=seed, task=task, output_directory=str(output_dir), min_speciation_rate = spec_rate)
                s.set_map_parameters(sample_file="null", 
                                     fine_file=str(fine_map_file), 
                                     sample_x=size,
                                     sample_y=size,
                                     fine_x=size,
                                     fine_y=size, 
                                     fine_x_offset=0,
                                     fine_y_offset=0,
                                     coarse_x=size*scale,
                                     coarse_y=size*scale,
                                     coarse_file = str(coarse_map_file),
                                     coarse_x_offset = offset,
                                     coarse_y_offset = offset,
                                     coarse_scale=1.0,
                                     historical_fine_map="none",
                                     historical_coarse_map="none")
                s.run()
                end = time.time()
                all_times.append({"time" : end-start, 
                                  "sigma" : sigma,
                                  "size" : size,
                                  "scale" : scale,
                                  "speciation_rate" : spec_rate,
                                  "seed" : seed})

In [14]:
output_df = pd.DataFrame(all_times)

In [15]:
output_csv = pathlib.Path(output_dir, "microbenchmarks.csv")
output_df.to_csv(output_csv)


In [16]:
shutil.copy2(output_csv, pathlib.Path("../rcoalescence_examples/output/bench/microbenchmarks.csv"))

PosixPath('../rcoalescence_examples/output/bench/microbenchmarks.csv')

### Now time on infinite landscapes (without quite so large a spatial scale so that the benchmarks complete quickly)

In [17]:
all_times_infinite = []
task = 1
for spec_rate in [0.00001, 0.001, 0.1]:
    for size in [2**x for x in range(3, 7)]:
        for sigma in [1, 2, 4, 8]:
            task += 1
            fine_map_file = pathlib.Path(temp_dir, "temp1.tif")
            create_null_tif(fine_map_file, size)
            for seed in range(1, 4):
                start = time.time()
                s = Simulation(logging_level=40)
                s.set_simulation_parameters(seed=seed, task=task + 100,
                                            output_directory=str(output_dir),
                                            min_speciation_rate = spec_rate,
                                            sigma=sigma, landscape_type="infinite")
                s.set_map_files("null", str(fine_map_file))
                s.run()
                end = time.time()
                all_times_infinite.append({"time" : end-start, 
                                           "sigma" : sigma,
                                           "size" : size,
                                           "scale" : "infinite",
                                           "speciation_rate" : spec_rate,
                                           "seed" : seed})


In [20]:
for spec_rate in [0.0000001, 0.00001, 0.001, 0.1]:
    for size in [2**x for x in range(8, 9)]:
        for sigma in [2]:
            task += 1
            fine_map_file = pathlib.Path(temp_dir, "temp1.tif")
            create_null_tif(fine_map_file, size)
            for seed in range(1, 4):
                start = time.time()
                s = Simulation(logging_level=40)
                s.set_simulation_parameters(seed=seed, task=task + 100,
                                            output_directory=str(output_dir),
                                            min_speciation_rate = spec_rate,
                                            sigma=sigma, landscape_type="infinite")
                s.set_map_files("null", str(fine_map_file))
                s.run()
                end = time.time()
                all_times_infinite.append({"time" : end-start, 
                                           "sigma" : sigma,
                                           "size" : size,
                                           "scale" : "infinite",
                                           "speciation_rate" : spec_rate,
                                           "seed" : seed})

In [21]:
infinite_df = pd.DataFrame(all_times_infinite)
infinite_csv = pathlib.Path(output_dir, "microbenchmarks_infinite.csv")
infinite_df.to_csv(infinite_csv)
shutil.copy2(infinite_csv, pathlib.Path("../rcoalescence_examples/output/bench/microbenchmarks_infinite.csv"))

PosixPath('../rcoalescence_examples/output/bench/microbenchmarks_infinite.csv')

In [22]:
infinite_df

,scale,seed,sigma,size,speciation_rate,time
0,infinite,1,1,8,1.000000e-05,0.324201
1,infinite,2,1,8,1.000000e-05,0.190786
2,infinite,3,1,8,1.000000e-05,0.111163
3,infinite,1,2,8,1.000000e-05,0.276390
4,infinite,2,2,8,1.000000e-05,0.377919
5,infinite,3,2,8,1.000000e-05,0.280799
6,infinite,1,4,8,1.000000e-05,0.526024
7,infinite,2,4,8,1.000000e-05,0.431564
8,infinite,3,4,8,1.000000e-05,0.619434
9,infinite,1,8,8,1.000000e-05,1.875290
